In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from dateutil.parser import parse

In [2]:
mpath = os.getcwd()
path = mpath+'/DAS output'
files = os.listdir(path)

In [3]:
cols = []
df = pd.DataFrame()
for f in files:
    if 'pivot' not in f:
        df1 = pd.read_csv(path+'/'+f)
        df = pd.concat([df,df1],axis=0)

In [4]:
df.to_csv('MetaPointsRaw.csv')

In [5]:
print(len(df))
df = df.drop_duplicates()
print(len(df))

8415314
8347020


In [6]:
dates = df['timestamp'].values
date = []
for d in dates:
    date.append(parse(d))
df['time']=date

In [7]:
df = df.query("name!='Module 07 - Status Codes - Max / Min Cell Voltage Location'")
df['core']=df['deviceName'].apply(lambda x: x[0:7])
df.loc[:,'first'] = df['name'].apply(lambda x: x.split('-')[0].strip())
df['second']=df['first'].apply(lambda x: x.split(' ')[0])

In [9]:
df.to_csv('MetaPointsRaw.csv')

KeyboardInterrupt: 

In [10]:
dfmod = df[df['second']=='Module']
dfmod.loc[:,'point']=dfmod['name'].apply(lambda x: x.split('-')[2].strip())
dfmod = dfmod.reset_index()
dfmod.drop(columns=['index'],inplace=True)
dfmod.reset_index(drop=True,inplace=True)

In [11]:
dfmod[dfmod['point']=='Rack Number']['value'].unique()

array([ 7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,  0.,  1.,
        2.,  3.,  4.,  5.,  6.])

In [15]:
dfmod.head()

,xid,dataSourceName,deviceName,name,timestamp,value,rendered,time,core,first,second,point
0,BallaratAR_DP_10db3ac3-3742-4998-bfbf-4a7cb6ec...,Ballarat Array,Core 03 Node 01 BMS,Module 01 - Analogs - Average Cell Temperature,2020-05-01T21:00:00.000-04:00,18.0,18.0 °C,2020-05-01 21:00:00-04:00,Core 03,Module 01,Module,Average Cell Temperature
1,BallaratAR_DP_10db3ac3-3742-4998-bfbf-4a7cb6ec...,Ballarat Array,Core 03 Node 01 BMS,Module 01 - Analogs - Average Cell Temperature,2020-05-01T21:00:06.000-04:00,18.7,18.7 °C,2020-05-01 21:00:06-04:00,Core 03,Module 01,Module,Average Cell Temperature
2,BallaratAR_DP_10db3ac3-3742-4998-bfbf-4a7cb6ec...,Ballarat Array,Core 03 Node 01 BMS,Module 01 - Analogs - Average Cell Temperature,2020-05-01T21:00:16.000-04:00,18.2,18.2 °C,2020-05-01 21:00:16-04:00,Core 03,Module 01,Module,Average Cell Temperature
3,BallaratAR_DP_10db3ac3-3742-4998-bfbf-4a7cb6ec...,Ballarat Array,Core 03 Node 01 BMS,Module 01 - Analogs - Average Cell Temperature,2020-05-01T21:00:26.000-04:00,19.0,19.0 °C,2020-05-01 21:00:26-04:00,Core 03,Module 01,Module,Average Cell Temperature
4,BallaratAR_DP_10db3ac3-3742-4998-bfbf-4a7cb6ec...,Ballarat Array,Core 03 Node 01 BMS,Module 01 - Analogs - Average Cell Temperature,2020-05-01T21:00:36.000-04:00,18.2,18.2 °C,2020-05-01 21:00:36-04:00,Core 03,Module 01,Module,Average Cell Temperature


In [40]:
dfmod1 = dfmod[['time','core','first','point','value']]
dfmod1.set_index(['time','core','first','point'],append=True,inplace=True)
df_mod = dfmod1.unstack(-1)
df_mod.columns = df_mod.columns.droplevel()
df_mod.reset_index(inplace=True)

In [57]:
df_mods = dfmod.pivot_table(index=['time','core','first'],columns='point',values='value').reset_index()
print(len(df_mods))

2912503


In [42]:
df_mod.head()

point,level_0,time,core,first,Average Cell Temperature,Max Cell Temperature,Min Cell Temperature,Rack Number
0,0,2020-05-01 21:00:00-04:00,Core 03,Module 01,18.0,NaN,NaN,NaN
1,1,2020-05-01 21:00:06-04:00,Core 03,Module 01,18.7,NaN,NaN,NaN
2,2,2020-05-01 21:00:16-04:00,Core 03,Module 01,18.2,NaN,NaN,NaN
3,3,2020-05-01 21:00:26-04:00,Core 03,Module 01,19.0,NaN,NaN,NaN
4,4,2020-05-01 21:00:36-04:00,Core 03,Module 01,18.2,NaN,NaN,NaN


In [43]:
df_mod.loc[:,'module'] = df_mod['first'].apply(lambda x: int(x[-2:]))
df_mod.drop(columns=['first','level_0'],inplace=True)

In [49]:
cols = df_mod.columns.tolist()
col = cols[0:2]+cols[-2:]+cols[2:-2]
df_mod = df_mod[col]
df_mod.rename(columns={'time':'Time','core':'Core','module':'Module'},inplace=True)
df_mod

point,Time,Core,Rack Number,Module,Average Cell Temperature,Max Cell Temperature,Min Cell Temperature
0,2020-05-01 21:00:00-04:00,Core 03,NaN,1,18.0,NaN,NaN
1,2020-05-01 21:00:06-04:00,Core 03,NaN,1,18.7,NaN,NaN
2,2020-05-01 21:00:16-04:00,Core 03,NaN,1,18.2,NaN,NaN
3,2020-05-01 21:00:26-04:00,Core 03,NaN,1,19.0,NaN,NaN
4,2020-05-01 21:00:36-04:00,Core 03,NaN,1,18.2,NaN,NaN
...,...,...,...,...,...,...,...
7960641,2020-05-03 11:59:24-04:00,Core 04,NaN,1,17.2,NaN,NaN
7960642,2020-05-03 11:59:30-04:00,Core 04,NaN,1,17.5,NaN,NaN
7960643,2020-05-03 11:59:34-04:00,Core 04,NaN,1,18.2,NaN,NaN
7960644,2020-05-03 11:59:40-04:00,Core 04,NaN,1,18.5,NaN,NaN


In [75]:
df_rack = df_mod[['Time','Core','Module','Rack Number']][df_mod['Rack Number'].isnull()==False]
df_mean = df_mod[['Time','Core','Module','Average Cell Temperature']][df_mod['Average Cell Temperature'].isnull()==False]
df_max = df_mod[['Time','Core','Module','Max Cell Temperature']][df_mod['Max Cell Temperature'].isnull()==False]
df_min = df_mod[['Time','Core','Module','Min Cell Temperature']][df_mod['Min Cell Temperature'].isnull()==False]

In [84]:
len(df_rack)+len(df_mean)#+len(df_max)+len(df_min)

4707985

In [80]:
print(len(df_rack),len(df_mean),len(df_max),len(df_min))

2760578 1947407 1681207 1571454


In [93]:
df_mod_temps = pd.merge(df_rack,df_mean,on=['Time','Core','Module'],how='outer')
df_mod_temps = pd.merge(df_mod_temps,df_max,on=['Time','Core','Module'],how='outer')
df_mod_temps = pd.merge(df_mod_temps,df_min,on=['Time','Core','Module'],how='outer')

In [105]:
df_mod_temps.dropna(subset=['Rack Number'],inplace=True)

In [106]:
df_mod_temps['Rack Number'].unique()

array([ 7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,  0.,  1.,
        2.,  3.,  4.,  5.,  6.])

In [107]:
dfcurr = df[df['second']=='Current']
dfcurr.loc[:,'sensor'] = dfcurr['first'].apply(lambda x: x.split(' ')[1]+'_'+x.split(' ')[3])
df_curr = dfcurr[['time','core','sensor','value']]
df_curr

/Users/alejandrajolodosky/.asdf/installs/python/3.7.7/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/alejandrajolodosky/.asdf/installs/python/3.7.7/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,time,core,sensor,value
0,2020-05-02 12:00:00-04:00,Core 03,Humidity_1,35.8
1,2020-05-02 12:00:02-04:00,Core 03,Humidity_1,35.9
2,2020-05-02 12:00:06-04:00,Core 03,Humidity_1,35.8
3,2020-05-02 12:00:08-04:00,Core 03,Humidity_1,35.9
4,2020-05-02 12:00:10-04:00,Core 03,Humidity_1,35.8
...,...,...,...,...
15052,2020-05-03 11:55:34.092000-04:00,Core 03,Temperature_8,21.0
15053,2020-05-03 11:56:36.141000-04:00,Core 03,Temperature_8,21.0
15054,2020-05-03 11:57:36.141000-04:00,Core 03,Temperature_8,21.0
15055,2020-05-03 11:58:36.142000-04:00,Core 03,Temperature_8,21.0


In [113]:
dfhvac = df[df['second']=='HVAC']
dfhvac['HVAC']=dfhvac['first'].apply(lambda x: x.split(' ')[2])
dfhvac['point']=dfhvac['first'].apply(lambda x: x.split(' ')[3]+'_'+x.split(' ')[4])
#df_hvac = dfhvac.pivot_table(index=['time','core','HVAC'],columns='point',values='value').reset_index()

/Users/alejandrajolodosky/.asdf/installs/python/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/alejandrajolodosky/.asdf/installs/python/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [114]:
dfhvac.reset_index(drop=True,inplace=True)
dfhvac1=dfhvac[['time','core','HVAC','point','value']]
dfhvac1.set_index(['time','core','HVAC','point'],append=True,inplace=True)
df_hvac = dfhvac1.unstack(-1)
df_hvac.columns = df_hvac.columns.droplevel()
df_hvac.reset_index(inplace=True)

In [117]:
df_hvac.drop(columns=['level_0'],inplace=True)
df_hvac.rename(columns={'time':'Time','core':'Core'},inplace=True)

In [120]:
df_cool1 = df_hvac[['Time','Core','HVAC','Cooling_1']][df_hvac['Cooling_1'].isnull()==False]
df_cool2 = df_hvac[['Time','Core','HVAC','Cooling_2']][df_hvac['Cooling_2'].isnull()==False]
df_hum = df_hvac[['Time','Core','HVAC','Current_Humidity']][df_hvac['Current_Humidity'].isnull()==False]
df_temp = df_hvac[['Time','Core','HVAC','Current_Temperature']][df_hvac['Current_Temperature'].isnull()==False]

In [122]:
df_HVAC = pd.merge(df_cool1,df_cool2,on=['Time','Core','HVAC'],how='outer')
df_HVAC = pd.merge(df_HVAC,df_hum,on=['Time','Core','HVAC'],how='outer')
df_HVAC = pd.merge(df_HVAC,df_temp,on=['Time','Core','HVAC'],how='outer')

In [125]:
#df_hvac.rename(columns={'time':'Time','core':'Core'},inplace=True)
df_HVAC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150465 entries, 0 to 150464
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype                                 
---  ------               --------------   -----                                 
 0   Time                 150465 non-null  datetime64[ns, tzoffset(None, -14400)]
 1   Core                 150465 non-null  object                                
 2   HVAC                 150465 non-null  object                                
 3   Cooling_1            8660 non-null    float64                               
 4   Cooling_2            8660 non-null    float64                               
 5   Current_Humidity     46874 non-null   float64                               
 6   Current_Temperature  103874 non-null  float64                               
dtypes: datetime64[ns, tzoffset(None, -14400)](1), float64(4), object(2)
memory usage: 9.2+ MB


In [88]:
df_mod.to_pickle('Module_Temps.pkl')
df_curr.to_pickle('Sensors.pkl')
df_hvac.to_pickle('HVAC.pkl')